In [252]:
import mnist_reader
import numpy as np
import random
import math
import pickle

from sklearn.model_selection import train_test_split

In [167]:
# Loading the test and train datasets.
x_train = np.array(mnist_reader.x_train)
x_test = np.array(mnist_reader.x_test)
y_train = np.array(mnist_reader.y_train)
y_test = np.array(mnist_reader.y_test)

In [168]:
# Reshaping to make the images of 28*28 size into a feature vector of size 784.
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2])

Parts 1, 2 and 3

In [1]:
class NeuralNetwork():
    def __init__(self, N, hidden_layer_sizes, lr, activation_fn, weight_init_fn, n_epochs, batch_size, early_stopping = False):
        # Error Handling
        if not isinstance(N, (int)):
            raise TypeError(f"No. of layers must be an integer. Instead received {type(N).__name__}.")
        if not isinstance(hidden_layer_sizes, (list)):
            raise TypeError(f"Hidden Layers have to be put in a list, not in a {type(hidden_layer_sizes).__name__}.")
        if not isinstance(lr, (int, float)):
            raise TypeError(f"Learning Rate must be a number. Instead received {type(N).__name__}.")
        if not isinstance(activation_fn, (str)):
            raise TypeError(f"Activation Function must be a string. Instead received {type(N).__name__}.")
        if not isinstance(weight_init_fn, (str)):
            raise TypeError(f"Weight Initialization Function must be a string. Instead received {type(N).__name__}.")
        if not isinstance(n_epochs, (int)):
            raise TypeError(f"No. of Epochs can only be an integer. Received instead {type(n_epochs).__name__}")
        if not isinstance(batch_size, (int)):
            raise TypeError(f"Batch size can only be an integer. Received instead {type(n_epochs).__name__}")
        if not isinstance(early_stopping, (bool)):
            raise TypeError(f"Early Stopping only accepts a boolean value. Instead received {type(early_stopping).__name__}.")

        if N <= 0:
            raise ValueError("No. of layers must be a positive number.")
        if lr < 0:
            raise ValueError("Learning Rate can't be negative!")
        if activation_fn not in ["sigmoid", "tanh", "relu", "leaky_relu"]:
            raise ValueError("Incorrect activation function. Activation function can only be from the following: sigmoid, tanh, relu, leaky_relu!")
        if weight_init_fn not in ["zero_init", "random_init", "normal_init"]:
            raise ValueError("Incorrect weight initialization function. It can only be from the following: zero_init, random_init, and normal_init!")
        if n_epochs <= 0:
            raise ValueError("No. of epochs has to be positive.")
        if batch_size <= 0:
            raise ValueError("Batch Size has to be positive!")
        
        self.N = N
        self.hidden_layer_sizes = hidden_layer_sizes
        self.lr = lr
        self.activation_fn = activation_fn
        self.weight_init_fn = weight_init_fn
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.early_stopping = early_stopping

        self.input_dim = 1 # Will be set later when we fit the data.
        self.has_fit = False # Indicates whether a fit has already been made or not.
        self.list_weights = [] # List of weights at each layer.
        self.list_biases = [] # List of biases after each layer.
        self.n_classes = 0 # No. of labels
        self.val_update_stage = 0 # Denotes the last occasion when the val loss improved.
        self.val_losses = [] # Array to store the validation losses per epoch
        self.train_losses = [] # Array to store the training losses per epoch
        self.best_val_loss = float('inf') # Best val loss seen so far during training.

    # Assuming that if it is called twice, on the second time, it will improve the pre-existing model, not start afresh.
    def fit(self, X, Y):
        if not self.has_fit:
            self.n_classes = len(np.unique(Y))
            self.init_weight(np.shape(X)[1])
            self.has_fit = True

        # Splitting X and Y into train and val into a 8:1 train:val ratio as mentioned in the question.
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 1/9, random_state = 42)
        self.train(X_train, Y_train, X_val, Y_val)

    def predict(self, X):
        if not self.has_fit:
            print("Model not trained yet.")
            return
        
        list_probs = self.return_probabilities(X)

        min_prob = 0
        curr_class = 0
        for i in range(self.n_classes):
            if list_probs[0, i] > min_prob:
                min_prob = list_probs[0, i]
                curr_class = i

        print(f"It belongs to class {curr_class}.")

        return curr_class
    
    def predict_proba(self, X):
        if not self.has_fit:
            print("Model not trained yet.")
            return
        
        data_point = X.copy()

        for k in range(self.N):
            data_point = np.dot(data_point, self.list_weights[k]) + self.list_biases[k]
            data_point = self.activate(data_point) # Putting the activation.

        data_point = np.dot(data_point, self.list_weights[self.N]) + self.list_biases[self.N]
        data_point = self.activate(data_point, softmax = True)

        for i in range(self.n_classes):
            print(f"Probability for Class {i} -> {data_point[0, i]}")

    def score(self, X, Y):
        if not self.has_fit:
            print("Model not trained yet.")
            return
        
        n_samples = np.shape(X)[0]
        n_correct = 0
        for i in range(n_samples):
            y_pred = self.predict_internal(X[i:i+1, :])
            if y_pred == Y[i]:
                n_correct += 1

        accuracy = round((n_correct / n_samples) * 100, 2)
        print(f"The accuracy is {accuracy}%.")

        return accuracy

    # Function that returns the array of probabilities for being each class for a datapoint.
    def return_probabilities(self, X):
        data_point = X.copy()

        for k in range(self.N):
            data_point = np.dot(data_point, self.list_weights[k]) + self.list_biases[k]
            data_point = self.activate(data_point) # Putting the activation.

        data_point = np.dot(data_point, self.list_weights[self.N]) + self.list_biases[self.N]
        data_point = self.activate(data_point, softmax = True)

        return data_point
    
    def predict_internal(self, X):
        if not self.has_fit:
            print("Model not trained yet.")
            return
        
        list_probs = self.return_probabilities(X)

        min_prob = 0
        curr_class = 0
        for i in range(self.n_classes):
            if list_probs[0, i] > min_prob:
                min_prob = list_probs[0, i]
                curr_class = i

        return curr_class

    # Trains the model based on the given X and Y.
    def train(self, X, Y, X_Val, Y_Val):
        # Making temporary bias and weight matrices for use later on.
        tmp_bias_arr = []
        tmp_weight_arr = []
        for i in self.list_weights:
            tmp_weight_arr.append(np.zeros(np.shape(i)))
        for i in self.list_biases:
            tmp_bias_arr.append(np.zeros(np.shape(i)))

        for iter_no in range(self.n_epochs):
            n_samples = np.shape(X)[0]

            n_batch = math.ceil(n_samples / self.batch_size)
            for i in range(n_batch):
                print(i)
                # Finding indices of first and last_samples in batch.
                init_sample = self.batch_size*i
                last_sample = min(self.batch_size*(i+1), n_samples) - 1
                data_samples = X[init_sample:last_sample+1, :].copy() 

                # Making an array of intermediary results that stores the activated values at each layer.
                intermed_arr = []
                intermed_arr.append(data_samples.copy())

                # Finding activations after each layer.
                for k in range(self.N):
                    data_samples = np.dot(data_samples, self.list_weights[k]) + self.list_biases[k]
                    data_samples = self.activate(data_samples) # Putting the activation.
                    intermed_arr.append(data_samples.copy())

                data_samples = np.dot(data_samples, self.list_weights[self.N]) + self.list_biases[self.N]
                data_samples = self.activate(data_samples, softmax = True)

                # Perform the backward Pass.
                self.backward_pass(data_samples, tmp_weight_arr, tmp_bias_arr, intermed_arr, Y[init_sample:last_sample+1])

                # Clearing the temporary arrays.
                intermed_arr.clear()
                del intermed_arr
                
                batchsize = last_sample - init_sample + 1
                tmp_weight_arr = [matrix / batchsize for matrix in tmp_weight_arr] # Finding average gradient for each weight by dividing by batchsize.
                tmp_bias_arr = [arr / batchsize for arr in tmp_bias_arr]

                # Updating the weights.
                for j in range(len(self.list_weights)):
                    self.list_weights[j] = self.list_weights[j] - self.lr * tmp_weight_arr[j]
                    self.list_biases[j] = self.list_biases[j] - self.lr * tmp_bias_arr[j]

                # Resetting the temporary arrays to fill them with zeroes again.
                for j in tmp_weight_arr:
                    j.fill(0)
                
                for j in tmp_bias_arr:
                    j.fill(0)

            n_val_samples = np.shape(X_Val)[0]

            train_loss = 0
            for i in range(n_samples):
                prob_list = self.return_probabilities(X[i:i+1, :])
                # print(prob_list)
                train_loss += ((-1) * np.log(prob_list[0, Y[i]])) # Loss is cross-entropy so -ve of log of the probability of the correct class appearing.

            val_loss = 0
            for i in range(n_val_samples):
                prob_list = self.return_probabilities(X_Val[i:i+1, :])
                # print(prob_list)
                val_loss += ((-1) * np.log(prob_list[0, Y_Val[i]]))

            self.train_losses.append(train_loss/np.shape(X)[0])
            self.val_losses.append(val_loss/np.shape(X_Val)[0])

            print(f"Epoch {iter_no + 1} -> Val Loss: {val_loss/np.shape(X_Val)[0]}, Training Loss: {train_loss/np.shape(X)[0]}.")

            # Implementing early stopping.
            if self.early_stopping == True:
                if iter_no == 0:
                    continue
                else:
                    if val_loss >= self.best_val_loss:
                        self.val_update_stage += 1
                    else:
                        self.val_update_stage = 0

                if self.val_update_stage == 5:
                    print("Early Stopping! Val Loss hasn't improved in the last 5 iterations!")
                    break

    # Performs the backward pass on a given "input", and adds the resultant gradient for each weight to the 'res_weights' object.
    def backward_pass(self, input, res_weights, res_bias, intermed_vals, y_actual):
        list_dims = [] # List of all the dimensions
        for i in self.hidden_layer_sizes:
            list_dims.append(i)
        list_dims.append(self.n_classes)

        local_grad = [] # List to store all the local gradients at each stage.
        for i in list_dims:
            local_grad.append(np.zeros((i, np.shape(input)[0])))

        # First, update weights attached to output layer.
        shape_wght = np.shape(self.list_weights[self.N])
        for i in range(shape_wght[1]):
            condition = (y_actual == i)
            condition = condition.astype(int)
            tmp = condition.reshape(np.shape(y_actual)[0], 1) # If y_actual == i, it is 1, else 0.

            local_gradient = input[:, i:i+1] - tmp # Gradient achieved after differentiating loss function of softmax wrt pre-activated input.
            local_gradient = local_gradient.T
            local_grad[self.N][i,:] = local_gradient

            # Weight Updation
            for j in range(shape_wght[0]):
                res_weights[self.N][j, i] += np.dot(local_gradient, intermed_vals[self.N][:,j:j+1])[0,0] # Local Gradient * y_i
            res_bias[self.N][0, i] += np.sum(local_gradient) # Bias where we are effectively just doing a dot product with a column vector of 1s (or sum).

        # Updating the other weights.
        for layer in range(self.N - 1, -1, -1):
            shape_wght = np.shape(self.list_weights[layer])

            for i in range(shape_wght[1]):
                for j in range(np.shape(self.list_weights[layer+1])[1]):
                    local_grad[layer][i,:] += local_grad[layer+1][j,:] * self.list_weights[layer+1][i, j]
                local_gradient = local_grad[layer][i,:]
                local_gradient = local_gradient.reshape(1,-1)
                
                # Weight updation
                for j in range(shape_wght[0]):
                    res_weights[layer][j, i] += np.dot(local_gradient, intermed_vals[layer][:,j:j+1] * self.activation_gradient(intermed_vals[layer+1][:,i:i+1]))[0,0]
                res_bias[layer][0, i] += np.dot(local_gradient, self.activation_gradient(intermed_vals[layer+1][:,i:i+1]))[0,0] # Bias

        # Clearing the temporary arrays.
        local_grad.clear()
        del local_grad

    # Initializes the weights and biases based on the weight init function -> Part 3.
    def init_weight(self, first_dim):
        list_dims = []
        list_dims.append(first_dim)
        for i in self.hidden_layer_sizes:
            list_dims.append(i)
        list_dims.append(self.n_classes)

        for i in range(len(list_dims) - 1):
            # Initializing with zeros.
            weight_matrix = np.zeros((list_dims[i], list_dims[i+1]))
            bias_arr = np.zeros((1, list_dims[i+1]))

            if self.weight_init_fn == 'zero_init':
                self.list_biases.append(bias_arr)
                self.list_weights.append(weight_matrix)

            # Randomly initializing weights. I am setting them between -0.1 and 0.1 to not keep them too large and close to 0.
            elif self.weight_init_fn == 'random_init':
                for j in range(list_dims[i]):
                    for k in range(list_dims[i+1]):
                        weight_matrix[j, k] = -0.1 + 0.2 * random.random() # Keeps it between -0.1 and 0.1.

                for j in range(list_dims[i+1]):
                    bias_arr[0, j] = -0.1 + 0.2 * random.random()

                self.list_biases.append(bias_arr)
                self.list_weights.append(weight_matrix)

            # Initializing weights by sampling from the normal distribution.
            elif self.weight_init_fn == 'normal_init':
                for j in range(list_dims[i]):
                    for k in range(list_dims[i+1]):
                        weight_matrix[j, k] = random.gauss(0, 1)

                for j in range(list_dims[i+1]):
                    bias_arr[0, j] = -0.1 + 0.2 * random.gauss(0, 1)

                self.list_biases.append(bias_arr)
                self.list_weights.append(weight_matrix)

    # Function for passing an input through the activation function to get an output. -> Part 2
    def activate(self, input, softmax = False):
        if softmax == True:
            tmp = np.exp(input)
            sums_arr = np.sum(tmp, axis=1, keepdims=True)
            return tmp / sums_arr # Softmax is basically e^ai / (e^a1 + e^a2 + ... + e^an) where n is the number of classes.
            
        else:
            if self.activation_fn == 'sigmoid':
                return 1 / (1 + np.exp(-input)) # Sigmoid is 1 / (1 + e^(-x))
            
            elif self.activation_fn == 'tanh':
                return (np.exp(input) - np.exp(-input)) / (np.exp(input) + np.exp(-input)) # Tanh is (e^x - e^-x) / (e^x + e^-x)

            elif self.activation_fn == 'relu':
                return np.maximum(input, 0) # ReLU is max(0, x)
            
            # Using coefficient of 0.01 for leaky_RELU
            else:
                return np.where(input > 0, input, 0.01 * input) # Leaky_ReLU is x if x > 0 and alpha * x if x <= 0
    
    # Function to find gradients of each activation function. For softmax, pred_bool is when the corresponding softmax node is the 
    # correct prediction or not, and softmax_sum is the sum of all the e^a_is.
    def activation_gradient(self, input, softmax = False, softmax_sum = 0, pred_bool = False):
        if softmax == True:
            if pred_bool == True:
                tmp = 1
            else:
                tmp = 0
            return (np.exp(input) / softmax_sum) - tmp
        
        else:
            if self.activation_fn == 'sigmoid':
                sigmoid = 1 / (1 + np.exp(-input))
                return sigmoid * (1 - sigmoid) # The derivative of sigmoid = sigmoid * (1-sigmoid)
            
            elif self.activation_fn == 'tanh':
                tanh = (np.exp(input) - np.exp(-input)) / (np.exp(input) + np.exp(-input))
                return 1 - tanh**2 # Derivative of tanh is sech^2x which is 1 - tanh^2x

            elif self.activation_fn == 'relu':
                return np.where(input > 0, 1, 0) # Differentiating x gives 1, and differentiating 0 with respect to x gives 0.
            
            # Using coefficient of 0.01 for leaky_RELU
            else:
                return np.where(input > 0, 1, 0.01) # Differentiating x gives 1. Differentiating alpha*x gives alpha.

In [2]:
# Preparing the Train and Test Data

random.seed(42) # Seed to maintain consistency in results.

train_indices = np.random.choice(x_train.shape[0], size=36000, replace=False)
X_train = x_train[train_indices]
Y_train = y_train[train_indices]

test_indices = np.random.choice(x_test.shape[0], size=4000, replace=False)
X_test = x_test[test_indices]
Y_test = y_test[test_indices]

NameError: name 'random' is not defined

In [199]:
# Preparing the Train and Test Data

random.seed(42) # Seed to maintain consistency in results.

train_indices = np.random.choice(x_train.shape[0], size=300, replace=False)
X_train = x_train[train_indices]
Y_train = y_train[train_indices]

test_indices = np.random.choice(x_test.shape[0], size=50, replace=False)
X_test = x_test[test_indices]
Y_test = y_test[test_indices]

In [219]:
model = NeuralNetwork(N = 4, hidden_layer_sizes=[256,128,64,32], lr=2e-3, activation_fn='relu',weight_init_fn='random_init', 
                      n_epochs=5, batch_size=128, early_stopping=True)

model.fit(X_train, Y_train)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
Epoch 1 -> Val Loss: 0.4327109730008531, Training Loss: 0.4231970243420181.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14

In [250]:
model.score(X_test, Y_test)

The accuracy is 93.03%.


93.03

In [221]:
model2 = NeuralNetwork(N = 4, hidden_layer_sizes=[256,128,64,32], lr=2e-3, activation_fn='sigmoid',weight_init_fn='zero_init', 
                      n_epochs=5, batch_size=128, early_stopping=True)

model2.fit(X_train, Y_train)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
Epoch 1 -> Val Loss: 2.302043481515408, Training Loss: 2.301838910299733.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1

In [233]:
model2.score(X_test, Y_test)

The accuracy is 11.6%.


11.6

In [251]:
model3 = NeuralNetwork(N = 4, hidden_layer_sizes=[256,128,64,32], lr=2e-3, activation_fn='sigmoid',weight_init_fn='normal_init', 
                      n_epochs=5, batch_size=128, early_stopping=True)

model3.fit(X_train, Y_train)
model3.score(X_test, Y_test)

0


C:\Users\Abhirup Das\AppData\Local\Temp\ipykernel_13640\1824881627.py:336: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-input)) # Sigmoid is 1 / (1 + e^(-x))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
Epoch 1 -> Val Loss: 1.6005304598139989, Training Loss: 1.6126249286212906.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1

57.25

In [ ]:
# Pickling the custom object
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)

PicklingError: Can't pickle <class '__main__.NeuralNetwork'>: it's not the same object as __main__.NeuralNetwork

In [ ]:
# Unpickling the custom object
with open('model.pkl', 'rb') as file:
    loaded_person = pickle.load(file)

print(loaded_person.score(X_test, Y_test))

FileNotFoundError: [Errno 2] No such file or directory: 'person.pkl'